<a href="https://colab.research.google.com/github/Engr-Genius/DATA-ANALYSIS-WITH-PYTHON/blob/main/3MTT_FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Customer Subscribing to a Term Deposit

## OBJECTIVE
1. Identify the key demographic factors that influence subscription.
2. Analyze financial patterns across customers.
3. Understand the behavioral factors that affect whether a customer will subscribe to a term deposit.
4. Develop insights that will support better marketing strategies and campaign targeting.

## Project Questions
1. What is the monthly trend of total transaction amount, and how does it compare to the previous month?
2. How does the average transaction amount differ across customer age groups?
3. Which customer locations show the highest growth in transaction amounts over time?
4. What is the distribution of high-value vs low-value customers based on account balance?
5. What time of the day has the highest average transaction amount?
6. What percentage of total transaction revenue is contributed by the top 10% of customers?
7. Do customers with higher account balances transact more frequently?
8. Which Customer Locations have the highest transaction frequency vs average account balance?
9. How does male vs female spending compare across different age bands?
10. How has the average transaction amount changed year-over-year?
11. What are the top 5 cities contributing to total revenue, and how do they trend monthly?
12. What is the median transaction amount compared to the mean—by location?
13. What is the proportion of small transactions (<500 INR) vs large transactions (>10,000 INR)?
14. Do customers in metro cities (Mumbai, Delhi, Pune) have different transaction patterns from non-metro cities?
15. How many customers increased their account balance over time?

In [7]:
import pandas as pd

In [9]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
import pandas as pd
# Loading Dataset
df = pd.read_csv ('/content/drive/MyDrive/project data.csv')

In [22]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


Monthly Trend of Total Transaction Amount

In [27]:

# Convert date column to datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

# Extract month and year
df['Month'] = df['TransactionDate'].dt.to_period('M')

# Group by month and calculate total transaction amount
monthly_trend = df.groupby('Month')['TransactionAmount (INR)'].sum().reset_index()

# Calculate month-to-month difference
monthly_trend['Change_from_Previous_Month'] = monthly_trend['TransactionAmount (INR)'].diff()

monthly_trend

,Month,TransactionAmount (INR),Change_from_Previous_Month
0,2016-01,6.208414e+07,NaN
1,2016-02,6.651735e+07,4.433208e+06
2,2016-03,7.560450e+07,9.087153e+06
3,2016-04,8.077037e+07,5.165871e+06
4,2016-05,6.744887e+07,-1.332150e+07
5,2016-06,7.656311e+07,9.114231e+06
6,2016-07,7.508947e+07,-1.473634e+06
7,2016-08,6.696936e+08,5.946041e+08
8,2016-09,2.418754e+08,-4.278182e+08
9,2016-10,7.851996e+07,-1.633554e+08


Average Transaction Amount by Age Group

In [29]:
import pandas as pd
import datetime

# Convert 'CustomerDOB' to datetime objects and handle potential errors
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], errors='coerce', dayfirst=True)

# Calculate age
df['age'] = (datetime.datetime.now().year - df['CustomerDOB'].dt.year)

# Create age groups
bins = [0, 25, 35, 45, 60, 100]
labels = ['18-25', '26-35', '36-45', '46-60', '60+']

df['Age_Group'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)

# Compute average transaction amount per age group using the correct column name
age_group_avg = df.groupby('Age_Group')['TransactionAmount (INR)'].mean().reset_index()

age_group_avg

/tmp/ipython-input-3071469383.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], errors='coerce', dayfirst=True)
/tmp/ipython-input-3071469383.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_group_avg = df.groupby('Age_Group')['TransactionAmount (INR)'].mean().reset_index()


,Age_Group,TransactionAmount (INR)
0,18-25,1839.919519
1,26-35,889.478889
2,36-45,1416.636196
3,46-60,2023.261787
4,60+,NaN


Locations with Highest Growth Over Time

In [31]:
import pandas as pd

# Convert date column to datetime
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

# Extract month
df['Month'] = df['TransactionDate'].dt.to_period('M')

# Group by location and month
location_trend = df.groupby(['CustLocation', 'Month'])['TransactionAmount (INR)'].sum().reset_index()

# Sort values
location_trend = location_trend.sort_values(['CustLocation', 'Month'])

# Calculate month-to-month growth per location
location_trend['Growth'] = location_trend.groupby('CustLocation')['TransactionAmount (INR)'].diff()

location_trend

,CustLocation,Month,TransactionAmount (INR),Growth
0,(154) BHASKOLA FARIDABAD,2016-01,95.00,NaN
1,(154) BHASKOLA FARIDABAD,2016-07,179.00,84.00
2,(154) BHASKOLA FARIDABAD,2016-08,4372.63,4193.63
3,(154) BHASKOLA FARIDABAD,2016-09,1248.72,-3123.91
4,(154) BHASKOLA FARIDABAD,2016-10,45.00,-1203.72
...,...,...,...,...
38724,ZUMARI TILAIYA,2016-03,633.00,NaN
38725,ZUMARI TILAIYA,2016-04,428.00,-205.00
38726,ZUMARI TILAIYA,2016-08,6574.02,6146.02
38727,ZUMARI TILAIYA,2016-12,815.00,-5759.02


Distribution of high-value vs low-value customers based on account balance.

In [33]:

# define cut points (tweak thresholds as needed)
bins = [ -1, 50000, 200000, 1e12 ]   # example: <50k low, 50k-200k medium, >200k high
labels = ['Low', 'Medium', 'High']
df['Balance_Group'] = pd.cut(df['CustAccountBalance'], bins=bins, labels=labels)

# Count customers per group (unique customers)
cust_bal = df.groupby('Balance_Group')['CustomerID'].nunique().reset_index()
cust_bal.columns = ['Balance_Group', 'Num_Customers']

# Also percent
cust_bal['Percent'] = 100 * cust_bal['Num_Customers'] / cust_bal['Num_Customers'].sum()

cust_bal

/tmp/ipython-input-1918409128.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cust_bal = df.groupby('Balance_Group')['CustomerID'].nunique().reset_index()


,Balance_Group,Num_Customers,Percent
0,Low,668185,70.692820
1,Medium,175779,18.597115
2,High,101231,10.710065


What time of day has the highest average transaction amount

In [35]:

# Extract hour from 'TransactionTime' column
# Assuming TransactionTime is in HHMMSS integer format (e.g., 143207 for 14:32:07)
df['hour'] = df['TransactionTime'].astype(str).str.zfill(6).str[:2].astype(int)

# Calculate average transaction amount by hour
hour_avg = df.groupby('hour')['TransactionAmount (INR)'].mean().reset_index().sort_values('hour')

# Find the hour(s) with the highest average transaction amount
top_hour = hour_avg.loc[hour_avg['TransactionAmount (INR)'].idxmax()]

hour_avg, top_hour

(    hour  TransactionAmount (INR)
 0      0              1345.305432
 1      1              1206.247719
 2      2              1129.762666
 3      3              1071.634225
 4      4               997.589079
 5      5               971.678338
 6      6              1045.564943
 7      7              1039.150456
 8      8              1036.881077
 9      9              1181.025827
 10    10              1265.007033
 11    11              1530.081177
 12    12              1694.084258
 13    13              1762.040360
 14    14              1806.127773
 15    15              1805.056349
 16    16              1893.350496
 17    17              1945.549068
 18    18              1837.898696
 19    19              1571.620099
 20    20              1507.622547
 21    21              1341.645198
 22    22              1201.004691
 23    23              1296.456308,
 hour                         17.000000
 TransactionAmount (INR)    1945.549068
 Name: 17, dtype: float64)

What percentage of total transaction revenue is contributed by the top 10% of customers?

In [37]:

cust_rev = df.groupby('CustomerID')['TransactionAmount (INR)'].sum().reset_index()
cust_rev = cust_rev.sort_values('TransactionAmount (INR)', ascending=False).reset_index(drop=True)

# determine top 10% cutoff
n_top = int(len(cust_rev) * 0.10) or 1
top_cust = cust_rev.head(n_top)

# percentages
total_revenue = cust_rev['TransactionAmount (INR)'].sum()
top_revenue = top_cust['TransactionAmount (INR)'].sum()
percent_top10 = 100 * top_revenue / total_revenue

percent_top10, n_top

(np.float64(63.22119173963833), 87935)

Do customers with higher account balances transact more frequently?

In [40]:

# Use customers' account balance (assumed one snapshot per customer) by taking max or last
# Use latest balance per customer if 'account_balance' appears per transaction
latest_balance = df.sort_values('TransactionDate').groupby('CustomerID').last().reset_index()[['CustomerID','CustAccountBalance']]

# transaction counts per customer
txn_counts = df.groupby('CustomerID')['TransactionAmount (INR)'].count().reset_index().rename(columns={'TransactionAmount (INR)':'txn_count'})

# merge
cust_activity = pd.merge(latest_balance, txn_counts, on='CustomerID', how='inner')

# correlation and grouped summary
corr = cust_activity['CustAccountBalance'].corr(cust_activity['txn_count'])
summary = cust_activity.groupby(pd.qcut(cust_activity['CustAccountBalance'], q=4, duplicates='drop'))['txn_count'].mean().reset_index()

corr, summary

/tmp/ipython-input-767390950.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = cust_activity.groupby(pd.qcut(cust_activity['CustAccountBalance'], q=4, duplicates='drop'))['txn_count'].mean().reset_index()


(np.float64(0.0011548763730932945),
         CustAccountBalance  txn_count
 0        (-0.001, 4720.18]   1.184638
 1      (4720.18, 16787.94]   1.183794
 2     (16787.94, 57561.49]   1.183056
 3  (57561.49, 115035495.1]   1.186577)

Which Customer Locations have the highest transaction frequency vs average account balance?

In [43]:

# transaction frequency per location
loc_freq = df.groupby('CustLocation')['TransactionAmount (INR)'].count().reset_index().rename(columns={'TransactionAmount (INR)':'txn_count'})

# average account balance per location
loc_balance = df.groupby('CustLocation')['CustAccountBalance'].mean().reset_index().rename(columns={'CustAccountBalance':'avg_account_balance'})

# merge
loc_summary = pd.merge(loc_freq, loc_balance, on='CustLocation').sort_values('txn_count', ascending=False)

loc_summary.head(30)

,CustLocation,txn_count,avg_account_balance
5219,MUMBAI,101997,154281.864562
5738,NEW DELHI,84143,192450.453458
770,BANGALORE,81330,102518.067872
3058,GURGAON,73594,177232.125498
2055,DELHI,70549,107557.075197
5833,NOIDA,32656,110244.372042
1591,CHENNAI,29840,89530.634858
6656,PUNE,25690,98689.569965
3367,HYDERABAD,22976,130806.950079
8310,THANE,21397,66744.825912


How does male vs female spending compare across different age bands?

In [45]:

bins = [0,25,35,45,60,120]
labels = ['18-25','26-35','36-45','46-60','60+']
df['age_band'] = pd.cut(df['age'], bins=bins, labels=labels, include_lowest=True)

# pivot: average transaction_amount by gender and age_band
gender_age = df.groupby(['age_band','CustGender'])['TransactionAmount (INR)'].mean().reset_index()
pivot = gender_age.pivot(index='age_band', columns='CustGender', values='TransactionAmount (INR)').fillna(0)

pivot

/tmp/ipython-input-2567303592.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gender_age = df.groupby(['age_band','CustGender'])['TransactionAmount (INR)'].mean().reset_index()


CustGender,F,M,T
age_band,,,
18-25,1739.387950,1882.393343,0.0
26-35,1037.520787,820.551728,0.0
36-45,1672.991995,1329.134522,0.0
46-60,2215.646492,1963.891153,0.0
60+,0.000000,0.000000,0.0


How has the average transaction amount changed year-over-year?

In [47]:

df['year'] = df['TransactionDate'].dt.year

# average transaction by year
yearly_avg = df.groupby('year')['TransactionAmount (INR)'].mean().reset_index().sort_values('year')

# YoY change (%)
yearly_avg['YoY_change_pct'] = yearly_avg['TransactionAmount (INR)'].pct_change() * 100

yearly_avg

,year,TransactionAmount (INR),YoY_change_pct
0,2016,1566.096079,NaN


Top 5 cities contributing to total revenue, and how they trend monthly

In [49]:

city_rev = df.groupby('CustLocation')['TransactionAmount (INR)'].sum().reset_index().sort_values('TransactionAmount (INR)', ascending=False)
top5_cities = city_rev.head(5)['CustLocation'].tolist()

# monthly revenue by city (filter top5)
df['month'] = df['TransactionDate'].dt.to_period('M')
monthly_city = df[df['CustLocation'].isin(top5_cities)].groupby(['CustLocation','month'])['TransactionAmount (INR)'].sum().reset_index()

# pivot for a wide view (optional)
monthly_city_pivot = monthly_city.pivot(index='month', columns='CustLocation', values='TransactionAmount (INR)').fillna(0)

top5_cities, monthly_city_pivot.head(20)

(['MUMBAI', 'NEW DELHI', 'BANGALORE', 'GURGAON', 'DELHI'],
 CustLocation    BANGALORE        DELHI      GURGAON       MUMBAI    NEW DELHI
 month                                                                        
 2016-01        5291284.71   4011630.50   4030674.77   5713404.01   6715804.79
 2016-02        4687899.33   3856578.52   5698476.36   7846262.99   6381929.56
 2016-03        6178565.40   4393722.90   5239155.84   8658681.06   6245829.79
 2016-04        6594820.18   4914752.65   5867779.74   8985225.93   8349409.03
 2016-05        4591826.43   3650482.82   4447169.84   7529684.14   5975111.58
 2016-06        7475541.63   4359564.98   4661774.71   9035146.36   6516112.68
 2016-07        6481582.43   4725648.85   4848569.56   8228473.61   7456317.05
 2016-08       46406011.82  43174882.15  45592959.69  73882475.50  64621447.76
 2016-09       16382664.02  17497666.65  14548001.66  23791699.04  23743886.50
 2016-10        5178421.17   5549687.80   5856155.06   7640139.78   7842

What is the median transaction amount compared to the mean—by location?

In [52]:

median_mean_loc = df.groupby('CustLocation')['TransactionAmount (INR)'].agg(
    mean_amount='mean',
    median_amount='median'
).reset_index()

median_mean_loc

,CustLocation,mean_amount,median_amount
0,(154) BHASKOLA FARIDABAD,540.031818,50.000
1,(BEFORE YMCA BLDG) CHENNAI,1200.000000,1200.000
2,(BENAKA MDTS) BANGALORE,163.907143,29.000
3,(BRINDA BAN ) KOLKATA,628.600000,200.000
4,(DT) HOSUR,1398.191818,300.000
...,...,...,...
9270,ZONE WHITEFIELD BANGALORE,2360.330976,1372.000
9271,ZOPADPATTI TEHSIL KURLA MUMBAI,7990.000000,7990.000
9272,ZUARINAGAR,1014.810000,1247.835
9273,ZUMARI TILAIYA,938.891111,633.000


Proportion of small transactions (<500 INR) vs large transactions (>10,000 INR)

In [54]:

# Define categories
df['txn_type'] = df['TransactionAmount (INR)'].apply(
    lambda x: 'Small (<500)' if x < 500
              else 'Large (>10000)' if x > 10000
              else 'Medium'
)

# Count and proportion
txn_counts = df['txn_type'].value_counts().reset_index()
txn_counts.columns = ['transaction_type', 'count']

txn_counts['percent'] = 100 * txn_counts['count'] / txn_counts['count'].sum()

txn_counts

,transaction_type,count,percent
0,Small (<500),539350,51.780218
1,Medium,478848,45.971732
2,Large (>10000),23416,2.248050


Do customers in metro cities (Mumbai, Delhi, Pune) have different transaction patterns from non-metro cities?

In [56]:

# Define metro and non-metro
metro_cities = ['MUMBAI', 'DELHI', 'PUNE']

df['City_Type'] = df['CustLocation'].apply(
    lambda x: 'Metro' if x in metro_cities else 'Non-Metro'
)

# Compare average transaction amount
city_type_compare = df.groupby('City_Type')['TransactionAmount (INR)'].agg(
    avg_amount='mean',
    median_amount='median',
    total_txns='count'
).reset_index()

city_type_compare

,City_Type,avg_amount,median_amount,total_txns
0,Metro,1608.996221,500.0,198236
1,Non-Metro,1556.012401,440.0,843378


How many customers increased their account balance over time?

In [58]:

# Sort by date so we can compare earlier and later balances
df_sorted = df.sort_values(['CustomerID', 'TransactionDate'])

# Get earliest and latest balance per customer
balance_change = df_sorted.groupby('CustomerID')['CustAccountBalance'].agg(
    first_balance='first',
    last_balance='last'
).reset_index()

# Check increase
balance_change['increased'] = balance_change['last_balance'] > balance_change['first_balance']

# Count how many increased
num_increased = balance_change['increased'].sum()
num_total = len(balance_change)
percent_increased = 100 * num_increased / num_total

num_increased, percent_increased

(np.int64(70718), np.float64(8.042003370640854))